In [2]:
import os, glob, torch, pandas as pd
from PIL import Image
from torchvision import transforms
import torch.nn as nn
import timm
from google.colab import drive; drive.mount('/content/drive')
import os, glob

Mounted at /content/drive


In [3]:
from google.colab import files
uploaded = files.upload()

Saving cars_test.rar to cars_test.rar


In [9]:
!unrar x cars_test.rar /content/cars_test/

Streaming output truncated to the last 5000 lines.
Extracting  /content/cars_test/cars_test/03043.jpg                        36%  OK 
Extracting  /content/cars_test/cars_test/03044.jpg                        36%  OK 
Extracting  /content/cars_test/cars_test/03045.jpg                        36%  OK 
Extracting  /content/cars_test/cars_test/03046.jpg                        36%  OK 
Extracting  /content/cars_test/cars_test/03047.jpg                        36%  OK 
Extracting  /content/cars_test/cars_test/03048.jpg                        36%  OK 
Extracting  /content/cars_test/cars_test/03049.jpg                        36%  OK 
Extracting  /content/cars_test/cars_test/03050.jpg                        36%  OK 
Extracting  /content/cars_test/cars_test/03051.jpg                        36%  OK 
Extracting  /content/cars_test/cars_test/03052.jpg                        36%  OK 
Extracting  /content/cars_tes

In [10]:
TEST_DIR     = "/content/cars_test"  # your unzipped images (scans recursively)
WEIGHTS_PATH = "/content/drive/MyDrive/checkpoints/swin/swin_multihead.pth"  # <- put your Drive path
OUT_CSV      = "/content/swin_test_preds.csv"
BACKBONE     = "swin_tiny_patch4_window7_224"
TAU = {"brand": 0.50, "model": 0.50, "year": 0.50}  # below -> -1

In [11]:
class MultiHeadSwin(nn.Module):
    def __init__(self, backbone_name, n_brand, n_model, n_year, pretrained=False):
        super().__init__()
        self.backbone = timm.create_model(backbone_name, pretrained=pretrained, num_classes=0, global_pool="avg")
        h = self.backbone.num_features
        self.head_brand = nn.Linear(h, n_brand)
        self.head_model = nn.Linear(h, n_model)
        self.head_year  = nn.Linear(h, n_year)
    def forward(self, x):
        f = self.backbone(x)
        return self.head_brand(f), self.head_model(f), self.head_year(f)

# 2) Load, strip DP prefix, and RENAME head keys
sd = torch.load(WEIGHTS_PATH, map_location="cpu")
sd = sd.get("state_dict", sd)
sd = {k.replace("module.", "", 1): v for k, v in sd.items()}

# rename: brand_head->head_brand, model_head->head_model, year_head->head_year
def rename_heads(d):
    out = {}
    for k, v in d.items():
        k = k.replace("brand_head", "head_brand").replace("model_head", "head_model").replace("year_head", "head_year")
        out[k] = v
    return out

sd = rename_heads(sd)

# 3) Infer class counts from renamed head weights
n_brand = sd["head_brand.weight"].shape[0]
n_model = sd["head_model.weight"].shape[0]
n_year  = sd["head_year.weight"].shape[0]

# 4) Build and load
BACKBONE = "swin_tiny_patch4_window7_224"
model = MultiHeadSwin(BACKBONE, n_brand, n_model, n_year, pretrained=False)
model.load_state_dict(sd, strict=True)
model.eval()


MultiHeadSwin(
  (backbone): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (layers): Sequential(
      (0): SwinTransformerStage(
        (downsample): Identity()
        (blocks): Sequential(
          (0): SwinTransformerBlock(
            (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=96, out_features=288, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=96, out_features=96, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path1): Identity()
            (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=96, out_features=384, bias=

In [12]:
tf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225)),
])

def pick(logits, tau):
    probs = torch.softmax(logits, dim=1).squeeze(0)  # [C]
    conf, pred = probs.max(0)
    return (int(pred) if float(conf) >= tau else -1), float(conf)

# collect image paths
exts = ("*.jpg","*.jpeg","*.png","*.bmp","*.webp")
paths = sum([glob.glob(os.path.join(TEST_DIR, "**", e), recursive=True) for e in exts], [])
print("images found:", len(paths))

# --- inference loop ---
rows = []
with torch.no_grad():
    for p in paths:
        x = tf(Image.open(p).convert("RGB")).unsqueeze(0)  # [1,3,224,224]
        lb, lm, ly = model(x)  # logits for 3 heads
        b, cb = pick(lb, TAU["brand"])
        m, cm = pick(lm, TAU["model"])
        y, cy = pick(ly, TAU["year"])
        rows.append({
            "path": p,
            "brand_id": b, "brand_conf": round(cb, 4),
            "model_id": m, "model_conf": round(cm, 4),
            "year_id":  y, "year_conf":  round(cy, 4),
        })

pd.DataFrame(rows).to_csv(OUT_CSV, index=False)
print(f"Saved -> {OUT_CSV}  | images: {len(paths)}")

images found: 8041
Saved -> /content/swin_test_preds.csv  | images: 8041


In [8]:
pd.DataFrame(rows).to_csv(r"C:\Users\KLP\Desktop\MLcourse\carimage\swin_test_preds.csv", index=False, encoding="utf-8")